In [1]:
import os, shutil
from keras.preprocessing import image
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
base_dir = '/Users/cb/Desktop/image processing/cats_and_dogs'

In [3]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [4]:
from keras.preprocessing.image import ImageDataGenerator

In [5]:
train_datagen = ImageDataGenerator(rescale= 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
                     train_dir,
                     target_size = (150,150),
                     batch_size = 20, 
                     class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [7]:
validation_generator = test_datagen.flow_from_directory(
                       validation_dir,
                       target_size = (150, 150),
                       batch_size = 20,
                       class_mode = 'binary')

Found 1000 images belonging to 2 classes.


In [8]:
for data_batch, labels_batch in train_generator:
       print('data batch shape:', data_batch.shape)
       print('labels batch shape:', labels_batch.shape)
       break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


In [9]:
import keras
import numpy as np
from keras import Model
from keras.applications import vgg16

In [10]:
input_shape = (150, 150,3)
vgg_model = vgg16.VGG16(include_top = False, weights = 'imagenet', input_shape = input_shape)

In [11]:
vgg_model.trainable = False 
for layer in vgg_model.layers:
    layer.trainable = False 

In [12]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout

In [14]:
last = vgg_model.output

In [15]:
x = Flatten()(last)
x = Dense(256, activation = 'relu')(x)
x = Dropout(0.5)(x)
preds = Dense(1, activation = 'sigmoid')(x)
model = Model(vgg_model.input, preds)

In [16]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [18]:
history = model.fit_generator(
         train_generator,
         steps_per_epoch = 100,
         epochs = 5,
         validation_data = validation_generator,
         validation_steps =50)

Epoch 1/5
100/100 [==============================] - 10567s 106s/step - loss: 0.7744 - acc: 0.7335 - val_loss: 0.3144 - val_acc: 0.8680
Epoch 2/5
100/100 [==============================] - 978s 10s/step - loss: 0.3879 - acc: 0.8375 - val_loss: 0.4585 - val_acc: 0.7860
Epoch 3/5
100/100 [==============================] - 992s 10s/step - loss: 0.2870 - acc: 0.8870 - val_loss: 0.2556 - val_acc: 0.9030
Epoch 4/5
100/100 [==============================] - 1005s 10s/step - loss: 0.2596 - acc: 0.8865 - val_loss: 0.2913 - val_acc: 0.8980
Epoch 5/5
100/100 [==============================] - 990s 10s/step - loss: 0.2385 - acc: 0.9030 - val_loss: 0.2578 - val_acc: 0.9010
